# Template specialization and Template instantiation

* Part I: Template specialization
* Part II: Template instantiation
* Part III: Source code organization (C++ Templates)

# Part I: Template specialization

# 1 motivation

In some cases, it isn’t possible or desirable for a template to define exactly the same code for any type. For example, you might wish to define a code path to be executed only if the type argument is a pointer, or a std::wstring, or a type derived from a particular base class. In such cases you can define a specialization of the template for that particular type.

When a user instantiates the template with that type, the compiler uses the specialization to generate the class, and for all other types, the compiler chooses the more general template. Specializations in which all parameters are specialized are complete specializations. If only some of the parameters are specialized, it is called a partial specialization.

In [1]:
# include <string>
#include <iostream>
using namespace std;

In [2]:
class MyClass
{
public:
    int num;
    std::wstring description;
};

In [3]:
template <typename K, typename V>
class MyMap{/*...*/};

// partial specialization for string keys
template<typename V>
class MyMap<string, V> {/*...*/};

MyMap<int, MyClass> classes; // uses original template
MyMap<string, MyClass> classes2; // uses the partial specialization

A template can have any number of specializations as long as each specialized type parameter is unique. Only class templates may be partially specialized. **All complete and partial specializations of a template must be declared in the same namespace as the original template.**

Class templates can be partially specialized, and the resulting class is still a template. Partial specialization allows template code to be partially customized for specific types in situations, such as:

* A template has multiple types and only some of them need to be specialized. The result is a template parameterized on the remaining types.

* A template has only one type, but a specialization is needed for pointer, reference, pointer to member, or function pointer types. The specialization itself is still a template on the type pointed to or referenced.

## 2 Example: Define partial specialization so one type is int

The following example defines a template class that takes pairs of any two types and then defines a partial specialization of that template class specialized so that one of the types is int. The specialization defines an additional sort method that implements a simple bubble sort based on the integer.

In [5]:
// partial_specialization_of_class_templates3.cpp
// compile with: /EHsc
template <class Key, class Value> class Dictionary {
   Key* keys;
   Value* values;
   int size;
   int max_size;
public:
   Dictionary(int initial_size) :  size(0) {
      max_size = 1;
      while (initial_size >= max_size)
         max_size *= 2;
      keys = new Key[max_size];
      values = new Value[max_size];
   }
   void add(Key key, Value value) {
      Key* tmpKey;
      Value* tmpVal;
      if (size + 1 >= max_size) {
         max_size *= 2;
         tmpKey = new Key [max_size];
         tmpVal = new Value [max_size];
         for (int i = 0; i < size; i++) {
            tmpKey[i] = keys[i];
            tmpVal[i] = values[i];
         }
         tmpKey[size] = key;
         tmpVal[size] = value;
         delete[] keys;
         delete[] values;
         keys = tmpKey;
         values = tmpVal;
      }
      else {
         keys[size] = key;
         values[size] = value;
      }
      size++;
   }

   void print() {
      for (int i = 0; i < size; i++)
         cout << "{" << keys[i] << ", " << values[i] << "}" << endl;
   }
};

// Template partial specialization: Key is specified to be int.
template <class Value> class Dictionary<int, Value> {
   int* keys;
   Value* values;
   int size;
   int max_size;
public:
   Dictionary(int initial_size) :  size(0) {
      max_size = 1;
      while (initial_size >= max_size)
         max_size *= 2;
      keys = new int[max_size];
      values = new Value[max_size];
   }
   void add(int key, Value value) {
      int* tmpKey;
      Value* tmpVal;
      if (size + 1 >= max_size) {
         max_size *= 2;
         tmpKey = new int [max_size];
         tmpVal = new Value [max_size];
         for (int i = 0; i < size; i++) {
            tmpKey[i] = keys[i];
            tmpVal[i] = values[i];
         }
         tmpKey[size] = key;
         tmpVal[size] = value;
         delete[] keys;
         delete[] values;
         keys = tmpKey;
         values = tmpVal;
      }
      else {
         keys[size] = key;
         values[size] = value;
      }
      size++;
   }

   void sort() {
      // Sort method is defined.
      int smallest = 0;
      for (int i = 0; i < size - 1; i++) {
         for (int j = i; j < size; j++) {
            if (keys[j] < keys[smallest])
               smallest = j;
         }
         swap(keys[i], keys[smallest]);
         swap(values[i], values[smallest]);
      }
   }

   void print() {
      for (int i = 0; i < size; i++)
         cout << "{" << keys[i] << ", " << values[i] << "}" << endl;
   }
};

int main() {
   Dictionary<const char*, const char*> dict(10);
   dict.print();
   dict.add("apple", "fruit");
   dict.add("banana", "fruit");
   dict.add("dog", "animal");
   dict.print();

   Dictionary<int, const char*> dict_specialized(10);
   dict_specialized.print();
   dict_specialized.add(100, "apple");
   dict_specialized.add(101, "banana");
   dict_specialized.add(103, "dog");
   dict_specialized.add(89, "cat");
   dict_specialized.print();
   dict_specialized.sort();
   cout << endl << "Sorted list:" << endl;
   dict_specialized.print();
};

In [6]:
main();

{apple, fruit}
{banana, fruit}
{dog, animal}
{100, apple}
{101, banana}
{103, dog}
{89, cat}

Sorted list:
{89, cat}
{100, apple}
{101, banana}
{103, dog}


# Part II: Template instantiation

When a function template is first called for each type, the compiler creates an instantiation. Each instantiation is a version of the templated function specialized for the type. This instantiation will be called every time the function is used for the type. If you have several identical instantiations, even in different modules, only one copy of the instantiation will end up in the executable file.(implicit instantiation.)

You can use explicit instantiation to create an instantiation of a templated class or function without actually using it in your code. Because this is useful when you are creating library (.lib) files that use templates for distribution, uninstantiated template definitions are not put into object (.obj) files.(explicit instantiation)

Template instantiation has two forms: explicit instantiation and implicit instantiation.

## 1 explicit instantiation

You can explicitly tell the compiler when it should generate a definition from a template. This is called explicit instantiation. Explicit instantiation includes two forms: explicit instantiation declaration and explicit instantiation definition.

### a explicit instantiation declaration

You can provide an explicit instantiation declaration for a template specialization if an explicit instantiation definition of the template exists in other translation units or later in the same file. If one translation unit contains the explicit instantiation definition, other translation units can use the specialization without having the specialization instantiated multiple times. The following example demonstrates this concept:

In [ ]:
//sample1.h:
template<typename T, T val>
union A{
    T foo();
};
extern template union A<int, 55>;
template<class T, T val>
T A<T,val>::foo(void){
    return val;
}

//sampleA.C"
#include "sample1.h"
template union A<int,55>;

//sampleB.C:
#include "sample1.h"
int main(void){
    return A<int, 55>().foo();
}

sampleB.C uses the explicit instantiation definition of A<int, 55>().foo() in sampleA.C.

### b Explicit instantiation definition

The following are examples of explicit instantiation definitions:

In [ ]:
template<class T> class Array { void mf(); };
template class Array<char>;      // explicit instantiation
template void Array<int>::mf();  // explicit instantiation

template<class T> void sort(Array<T>& v) { }
template void sort(Array<char>&); // explicit instantiation definition

namespace N {
   template<class T> void f(T&) { }
}

template void N::f<int>(int&);
// The explicit instantiation definition is in namespace N.

int* p = 0;
template<class T> T g(T = &p);
template char g(char);                  // explicit instantiation definition

template <class T> class X {
   private:
      T v(T arg) { return arg; };
};

template int X<int>::v(int);    // explicit instantiation definition

template<class T> T g(T val) { return val;}
template<class T> void Array<T>::mf() { }

An explicit instantiation definition is an instantiation of a template specialization or a its member

* An explicit instantiation definition of a template is in the same namespace where you define the template.
* Access checking rules do not apply to the arguments in the explicit instantiation definitions. 
* Template arguments in an explicit instantiation definition can be private types or objects. In this example, you can use the explicit instantiation definition template int X<int>::v(int) even though the member function is declared to be private.
* The compiler does not use default arguments when you explicitly instantiate a template. In this example, you can use the explicit instantiation definition template char g(char) even though the default argument is an address of the type int.

##  implicit instantiation.

Unless a template specialization has been explicitly instantiated or explicitly specialized, the compiler will generate a specialization for the template only when it needs the definition. This is called implicit instantiation.

    https://www.ibm.com/docs/en/i/7.3?topic=only-implicit-instantiation-c

**The act of creating a new definition of a function, class, or member of a class from a template declaration and one or more template arguments is called template instantiation. The definition created from a template instantiation to handle a specific set of template arguments is called a specialization.**

# Part III: Source code organization (C++ Templates)

When defining a class template, you must organize the source code in such a way that the member definitions are visible to the compiler when it needs them. You have the choice of using the inclusion model or the explicit instantiation model. 

In the inclusion model, you include the member definitions in every file that uses a template. This approach is simplest and provides maximum flexibility in terms of what concrete types can be used with your template. Its disadvantage is that it can increase compilation times. The times can be significant if a project or the included files themselves are large. 

With the explicit instantiation approach, the template itself instantiates concrete classes or class members for specific types. This approach can speed up compilation times, but it limits usage to only those classes that the template implementer has enabled ahead of time. 

In general, we recommend that you use the inclusion model unless the compilation times become a problem.

## motivation

Templates aren't like ordinary classes in the sense that the compiler doesn't generate object code for a template or any of its members. There's nothing to generate until the template is instantiated with concrete types. When the compiler encounters a template instantiation such as MyClass<int> mc; and no class with that signature exists yet, it generates a new class. It also attempts to generate code for any member functions that are used. (implicit instantiation)
    
If those definitions are in a file that isn't #included, directly or indirectly, in the .cpp file that is being compiled, the compiler can't see them. From the compiler's point of view, it's not necessarily an error. The functions may be defined in another translation unit where the linker will find them. If the linker doesn't find that code, it raises an unresolved external error.

## solution 1: The inclusion model

The simplest and most common way to make template definitions visible throughout a translation unit, is to put the definitions in the header file itself. Any .cpp file that uses the template simply has to #include the header. This approach is used in the Standard Library.

In [ ]:
// in xxx.h
#include <iostream>

template<typename T, size_t N>
class MyArray
{
    T arr[N];
public:
    // Full definitions:
    MyArray(){}
    void Print()
    {
        for (const auto v : arr)
        {
            std::cout << v << " , ";
        }
    }

    T& operator[](int i)
   {
       return arr[i];
   }
};

With this approach, the compiler has access to the complete template definition and can instantiate templates on-demand for any type. It's simple and relatively easy to maintain. However, the inclusion model does have a cost in terms of compilation times. This cost can be significant in large programs, especially if the template header itself #includes other headers. Every .cpp file that #includes the header will get its own copy of the function templates and all the definitions. The linker will generally be able to sort things out so that you don't end up with multiple definitions for a function, but it takes time to do this work. In smaller programs that extra compilation time is probably not significant.

## solution 2: The explicit instantiation model

If the inclusion model isn't viable for your project, and you know definitively the set of types that will be used to instantiate a template, then you can separate out the template code into an .h and .cpp file, and in the .cpp file explicitly instantiate the templates. This approach generates object code that the compiler will see when it encounters user instantiations.

You create an explicit instantiation by using the keyword template followed by the signature of the entity you want to instantiate. This entity can be a type or a member. If you explicitly instantiate a type, all members are instantiated.

The header file MyArray.h declares template class MyArray:

In [ ]:
//MyArray.h
#ifndef MYARRAY
#define MYARRAY

template<typename T, size_t N>
class MyArray
{
    T arr[N];
public:
    MyArray();
    void Print();
    T& operator[](int i);
};
#endif

The source file MyArray.cpp explicitly instantiates template MyArray<double, 5> and template MyArray<string, 5>:

In [ ]:
//MyArray.cpp
#include <iostream>
#include "MyArray.h"

using namespace std;

template<typename T, size_t N>
MyArray<T,N>::MyArray(){}

template<typename T, size_t N>
void MyArray<T,N>::Print()
{
    for (const auto v : arr)
    {
        cout << v << "'";
    }
    cout << endl;
}

template MyArray<double, 5>;
template MyArray<string, 5>;

In the previous example, the explicit instantiations are at the bottom of the .cpp file. A MyArray may be used only for double or String types.

# Reference
* https://www.ibm.com/docs/en/i/7.3?topic=only-template-instantiation-c
* https://docs.microsoft.com/en-us/cpp/cpp/source-code-organization-cpp-templates?view=msvc-170